In [3]:
using JuMP
using GLPK

Antes de mais nada, a seguinte convenção

$$\begin{array}{r|l}\mbox{Raw carrots} & c_1\\ \mbox{Baked potatoes} & c_2 \\ \mbox{Wheat bread} & c_3 \\ \mbox{Cheddar cheese} & c_4 \\ \mbox{Peanut butter} & c_5\end{array}$$

para a quantidade de cada ingrediente na dieta.

Queremos resolver o seguinte problema:
$$\min \sum_{i=1}^5 \mbox{Cost}_i(c_i)$$

sujeito a:

$$\sum_{i=1}^5 \mbox{Calories}(c_i) \geq 0\\
\sum_{i=1}^5 \mbox{Fat}(c_i) \geq 50\\
\sum_{i=1}^5 \mbox{Protein}(c_i) \geq 100\\
\sum_{i=1}^5 \mbox{Carbohydrates}(c_i) \geq 250\\
c_i \geq 0 
$$

In [3]:
cost(i) = [0.14, 0.12,0.2,0.75,0.15][i]
calories(i) = [23, 171, 65, 112, 188,0,cal][i]
fat(i) = [0.1, 0.2,0,9.3,16,50,fat][i]
protein(i) = [0.6, 3.7, 2.2, 7,7.7,100,prot][i]
carbo(i) = [6,30,13,0,2,250,carb][i]

carbo (generic function with 1 method)

In [13]:
model = Model(GLPK.Optimizer)
@variable(model, 0 <= c1)
@variable(model, 0 <= c2)
@variable(model, 0 <= c3)
@variable(model, 0 <= c4)
@variable(model, 0 <= c5)

@objective(model,Min,cost(1)*c1+cost(2)*c2+cost(3)*c3+cost(4)*c4+cost(5)*c5)

for func in [calories, fat, protein, carbo]
    @constraint(model,func(1)*c1+func(2)*c2+func(3)*c3+func(4)*c4+func(5)*c5 >= func(6))
end

In [23]:
optimize!(model)
objective_value(model)

2.317754919499105

In [25]:
[value(c1),value(c2),value(c3),value(c4),value(c5)]

5-element Array{Float64,1}:
 0.0              
 7.714669051878355
 0.0              
 0.0              
 9.279964221824685

# Exercício 2

Antes de mais nada, a seguintes convenções:

$$\begin{array}{|r|l|}\hline\mbox{Produto} & \mbox{Quantidade produzida}\\ \hline\mbox{Skimmed milk} & p_1\\ \mbox{2%} & p_2 \\ \mbox{Whole milk} & p_3 \\ \mbox{Table cream} & p_4 \\ \mbox{Whipping cream} & p_5\\ \hline\end{array}$$

para a quantidade de cada produto produzido.

De início, vale notar que temos, efetivamente: $$500\mathcal{l} \times (1-0.037) + 250\mathcal{l} \times (1-0.049) = 719.25\mathcal{l}$$

de leite. Por conseguinte, temos:
$$500\mathcal{l} \times 0.037 + 250\mathcal{l} \times 0.049 = 30.75\mathcal{l}$$

de gordura totais.

Parece ser conveniente fazermos contas análogas da quantidade efetiva de leite e de gordura que cada produto irá precisar.

$$\begin{array}{|r|c|c|l|}\hline\mbox{Produto}& \mbox{Gordura (l)} & \mbox{Leite (l)} & \mbox{Profit (\$)}\\ \hline
p_1 & 0 & 2 & 0.1 \\ p_2 & 0.04 & 1.96 & 0.15  \\ p_3 & 0.08 & 1.92 & 0.2 \\ p_4 & 0.09 & 0.51 & 0.5 \\ p_5 & 0.135 & 0.165 & 1.2\\ \hline\end{array}$$ 

**Tornando nossa notação mais rigorosa.** 

Seja $\mathbf{p}\in \mathbb{R}^5$ a quantidade produzida de cada produto, $\mathbf{s}=\left[0.1\ 0.15\ 0.2\ 0.5\ 1.2\right]$ o lucro unitário para cada produto, $\mathbf{g}=\left[0\ 0.04\ 0.08\ 0.09\ 0.135\right]$ a gordura necessária para uma unidade de cada produto e $\mathbf{m}=\left[2\ 1.96\ 1.92\ 0.51\ 1.65\right]$ a quantidade de leite necessária para uma unidade de cada produto. 

Queremos resolver o seguinte problema:
$$\max_{\mathbf{p}} \mathbf{p}^\intercal\mathbf{s}$$

sujeito a:

$$\mathbf{p}^\intercal \mathbf{g} \leq 30.75\\ \mathbf{p}^\intercal \mathbf{m} \leq 719.25 \\ \mathbf{p} \geq 0$$

In [18]:
g = [0, 0.04, 0.08, 0.09, 0.135]
s = [0.1, 0.15, 0.2, 0.5, 1.2]
m = [2, 1.96, 1.92, 0.51, 1.65]

model_ex2 = Model(GLPK.Optimizer)

@variable(model_ex2, p[1:5] >= 0)
@constraint(model_ex2, sum(p.*g) <= 30.75)
@constraint(model_ex2, sum(p.*m) <= 719.25)

@objective(model_ex2, Max, sum(p.*s))

0.1 p[1] + 0.15 p[2] + 0.2 p[3] + 0.5 p[4] + 1.2 p[5]

In [19]:
optimize!(model)
objective_value(model)

0.0

In [20]:
p

5-element Array{VariableRef,1}:
 p[1]
 p[2]
 p[3]
 p[4]
 p[5]